In [40]:
!pip install -q langchain langchain-community langchain-groq


In [41]:
import os

os.environ["GROQ_API_KEY"] = "ENTER_YOUR_API_KEY"


In [42]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

In [43]:
#Priority: High ⬇
#email_subject = "Refund not received"
#email_body = "I applied for a refund 5 days ago but I have not received it yet."

#Priority: Medium ⬇
#email_subject = "Query about refund timeline"
#email_body = "Hi, I just wanted to know how many days it usually takes to process a refund. Thanks."

#Priority: Low ⬇
email_subject = "General feedback"
email_body = "I just wanted to say that I like your service and found it easy to use. Thank you!"


email_text = f"""
Subject: {email_subject}
Body: {email_body}
"""

In [44]:
from langchain_core.prompts import PromptTemplate

intent_prompt = PromptTemplate.from_template("""
Classify the intent of the email into ONE category:
Support, Complaint, Sales, Feedback, Spam.

Email:
{email}

Return only the category.
""")

intent = llm.invoke(intent_prompt.format(email=email_text)).content.strip()


In [45]:
if intent in ["Complaint", "Support"]:
    priority = "High"
elif intent == "Sales":
    priority = "Medium"
else:
    priority = "Low"


In [46]:
#CREATING_RAG

knowledge_base = [
    "Refunds are processed within 2 to 4 working days.",
    "If refund is delayed beyond 5 days, contact support team.",
    "Support hours are Monday to Friday, 10 AM to 5 PM.",
    "High priority issues are escalated to human support."
]

context = "\n".join(
    [k for k in knowledge_base if "refund" in email_text.lower()]
)

print("Retrieved Info:")
print(context)


In [47]:
reply_prompt = PromptTemplate.from_template("""
Use the information below to write a polite and professional reply.
Add my name "YOUR_NAME" in regards.
Use greeting "Dear Customer"
Insert line breaks

Information:
{context}

Email:
{email}

Reply:
""")

reply = llm.invoke(
    reply_prompt.format(context=context, email=email_text)
).content


In [48]:
if priority == "High":
    action = "Escalate to human support"
else:
    action = "Auto-reply"


In [49]:
print("\n📧 EMAIL TRIAGE RESULT")
print("---------------------")
print("Intent   :", intent)
print("Priority :", priority)
print("Action   :", action)

print("\n✉️ Generated Reply:\n")
print(reply)
